In [6]:
import rasterio
from rasterio.features import shapes
import geopandas as gp
from shapely.geometry import Point

outpoly = "D:/cbr/VP2/VP2_SoilMapping/polygone.shp"
naip_fn = 'D:/cbr/VP2/VP2_SoilMapping/Processed/Wetzwil_Ortho/prediction2/Prediction.tif'

mask = None
with rasterio.Env():
    with rasterio.open(naip_fn) as src:
        image = src.read(1) # first band
        results = (
        {'properties': {'raster_val': v}, 'geometry': s}
        for i, (s, v) 
        in enumerate(
            shapes(image, mask=mask, transform=src.transform)))


geoms = list(results)
 # first feature
#print(geoms[0])

gpd_polygonized_raster  = gp.GeoDataFrame.from_features(geoms)
print(gpd_polygonized_raster)

#gpd_polygonized_raster['geometry'] = gpd_polygonized_raster.apply(lambda x: Point((float(x.lon), float(x.lat))), axis=1)
gpd_polygonized_raster.to_file(outpoly, driver='ESRI Shapefile')

                                            geometry  raster_val
0  POLYGON ((2652012.080 1230016.603, 2652014.866...         2.0
1  POLYGON ((2651954.111 1229939.986, 2651955.235...         2.0
2  POLYGON ((2652181.447 1230185.881, 2652181.447...     -9999.0
3  POLYGON ((2651764.971 1230185.881, 2651764.971...     -9999.0
4  POLYGON ((2651803.123 1229808.141, 2651803.303...         2.0
5  POLYGON ((2651764.971 1229825.666, 2651764.971...     -9999.0
6  POLYGON ((2652180.728 1230185.881, 2652181.447...         1.0
7  POLYGON ((2652424.017 1229883.725, 2652424.062...     -9999.0


In [26]:
from osgeo import gdal, ogr, os

outpoly = "D:/cbr/VP2/VP2_SoilMapping/polygone.shp"
naip_fn = 'D:/cbr/VP2/VP2_SoilMapping/Processed/Wetzwil_Ortho/prediction2/Prediction.tif'

sourceRaster = gdal.Open(naip_fn)
band = sourceRaster.GetRasterBand(1)
bandArray = band.ReadAsArray()
driver = ogr.GetDriverByName("ESRI Shapefile")
if os.path.exists(outpoly):
    driver.DeleteDataSource(outpoly)
outDatasource = driver.CreateDataSource(outpoly)
outLayer = outDatasource.CreateLayer("polygonized", srs=None)
newField = ogr.FieldDefn('MYFLD')
outLayer.CreateField(newField)
gdal.Polygonize(band, None, outLayer, 0, [], callback=None )
outDatasource.Destroy()
sourceRaster = None
